In [1]:
import nltk
import pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import string 

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv('SMSSpamCollection.tsv', sep='\t')
data.columns = ['label', 'body_text']

def count_punch(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text)-text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punch%'] = data['body_text'].apply(lambda x: count_punch(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

#TF-IDF implementation
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punch%'], pd.DataFrame(X_tfidf.toarray())], axis = 1)

#CountVectorizer 

count_vect = CountVectorizer(analyzer=clean_text)
X_count  = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punch%'], pd.DataFrame(X_count.toarray())], axis = 1)

X_count_feat.head()

,body_len,punch%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
data['label']

0       spam
1        ham
2        ham
3        ham
4        ham
        ... 
5562    spam
5563     ham
5564     ham
5565     ham
5566     ham
Name: label, Length: 5567, dtype: object

## Exploring parameters setting using GridsearchCv

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [9]:
#from sklearn.preprocessing import LabelEncoder

#le = LabelEncoder()
#data['label'] = le.fit_transform(data['label']) 

In [10]:
data['label']

0       spam
1        ham
2        ham
3        ham
4        ham
        ... 
5562    spam
5563     ham
5564     ham
5565     ham
5566     ham
Name: label, Length: 5567, dtype: object

In [11]:
rf = RandomForestClassifier()
param = {
    'n_estimators': [10, 150, 300],
    'max_depth': [30, 60, 90, None]
}

gs = GridSearchCV(rf, param, cv= 5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending= False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,21.518667,0.532522,0.324681,0.039838,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.979354,0.976661,0.975741,0.966757,0.971249,0.973952,0.004445,1
10,26.703162,0.770185,0.220094,0.039808,None,150,"{'max_depth': None, 'n_estimators': 150}",0.977558,0.974865,0.972147,0.968553,0.973944,0.973414,0.002993,2
8,45.321100,0.360531,0.348034,0.017488,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.976661,0.976661,0.974843,0.967655,0.971249,0.973414,0.003492,3
11,40.410151,1.518256,0.278485,0.012555,None,300,"{'max_depth': None, 'n_estimators': 300}",0.975763,0.974865,0.973944,0.969452,0.971249,0.973055,0.002351,4
6,2.015076,0.289983,0.124825,0.013788,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.977558,0.973968,0.968553,0.965858,0.975741,0.972336,0.004423,5


In [12]:
rf = RandomForestClassifier()
param = {
    'n_estimators':[10, 150, 300],
    'max_depth':[30, 60, 90, None]
}

gs = GridSearchCV(rf, param, cv= 5, n_jobs=-1)
gs_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending= False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
8,47.236520,0.703477,0.433300,0.039418,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.978456,0.974865,0.974843,0.969452,0.970350,0.973593,0.003301,1
7,23.774477,0.340977,0.557714,0.375494,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.976661,0.973070,0.975741,0.969452,0.968553,0.972695,0.003250,2
10,26.324273,0.385562,0.285381,0.017894,None,150,"{'max_depth': None, 'n_estimators': 150}",0.976661,0.973968,0.973944,0.966757,0.969452,0.972156,0.003556,3
11,39.745893,0.719291,0.263503,0.018326,None,300,"{'max_depth': None, 'n_estimators': 300}",0.976661,0.973070,0.974843,0.966757,0.968553,0.971977,0.003750,4
3,6.696234,6.240609,0.510837,0.290491,60,10,"{'max_depth': 60, 'n_estimators': 10}",0.973968,0.967684,0.973944,0.969452,0.968553,0.970720,0.002700,5
